In [62]:
import pandas as pd
import numpy as np

In [63]:
df = pd.read_csv("RPGF3Results.csv")
df = df[df['Category: Developer Ecosystem'] == 1]
df.shape

(305, 58)

In [64]:
df.drop(columns=['Project_ID', 'Result: Quorum Reached', 'Meta: Applicant Type','Meta: Website','Meta: Bio','Meta: Payout Address',	'Category: Collective Governance','Category: Developer Ecosystem','Category: End User Experience and Adoption','Category: OP Stack','Keywords: Base','Keywords: Farcaster','Keywords: Zora','Link: Contract on Base','Link: Contract on OP Mainnet','Link: Dune','Link: Flipside','Link: GitHub','Link: GitHub (duneanalytics)','Link: GitHub (ethereum)','Link: GitHub (ethereum-optimism)','Link: NPM Package','Link: Optimism Gov','Link: Substack','Link: Twitter','GTP: VC Funding Amount','GTP: Has Token', 'GTP: Has VC Funding','Meta: Project Image'], inplace=True)

In [65]:
df_SVR = df[['Result: # Ballots', 'Meta: Project Name' ,'Result: Received OP','OSO: # GitHub Repos','OSO: Total Stars','OSO: Total Forks','OSO: Total Contributors','OSO: Contributors Last 6 Months','OSO: Avg Monthly Active Devs Last 6 Months', 'OSO: Total Onchain Users', 'OSO: Onchain Users Last 6 Months', 'OSO: Total Downloads', 'OSO: Downloads Last 6 Months']]	
df_SVR.fillna(0, inplace=True)
# df_SVR.reset_index(inplace=True)
df_SVR.to_csv("RetroDistribution.csv")

/var/folders/rg/45gtrwls705094nr5_p2f7980000gn/T/ipykernel_58679/2593781643.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_SVR.fillna(0, inplace=True)


## Normalization Calculation

## 

In [66]:
data = pd.read_csv("RetroDistribution.csv")
data.drop(columns=['Unnamed: 0'], inplace=True)

def normalize_data(data):
    normalized_data = {}
    for feature, values in data.items():
        if feature == "Meta: Project Name":
            continue 
        if all(isinstance(v, int) for v in values): 
            total_sum = sum(values)
            normalized_data[feature] = [v / total_sum for v in values]
        else: 
            try:
                values = [float(v) for v in values]  # Convert values to float
                max_value = max(values)
                normalized_data[feature] = [v / max_value for v in values]
            except ValueError:
                print(f"Error: Non-numeric value encountered in feature '{feature}'. Skipping normalization.")
    return normalized_data

def calculate_received_op(data):
    received_op = []
    for i in range(len(data)):
        received_op_value = data['OSO: Total Stars'][i] * data['OSO: Total Forks'][i] * data['OSO: Total Contributors'][i]
        received_op.append(received_op_value)
    return received_op

def adjust_scaling_factor(data, received_op, total_pool_size=1000000):
    scaling_factors = [1] * len(data) 
    total_contributions = data['OSO: Total Stars'] + data['OSO: Total Forks'] + data['OSO: Total Contributors']
    
    project_proportions = total_contributions / sum(total_contributions)
    
    adjusted_scaling_factors = [scaling_factor * proportion for scaling_factor, proportion in zip(scaling_factors, project_proportions)]
    
    scaled_received_op = [value * scaling_factor for value, scaling_factor in zip(received_op, adjusted_scaling_factors)]
    
    return scaled_received_op

def scale_received_op(received_op, total_pool_size=1000000):
    total_received_op = sum(received_op)
    scaling_factor = total_pool_size / total_received_op if total_received_op > 0 else 0
    scaled_received_op = [value * scaling_factor for value in received_op]
    return scaled_received_op

normalized_data = normalize_data(data.copy()) 
received_op = calculate_received_op(data)
scaled_received_op = scale_received_op(received_op)

for project, scaled_value in zip(data["Meta: Project Name"], scaled_received_op):
    print(f"Project: {project}, Scaled Result: Received OP: {scaled_value:.2f}")


Project: Protocol Guild, Scaled Result: Received OP: 704014.50
Project: go-ethereum, Scaled Result: Received OP: 217310.60
Project: Solidity, Scaled Result: Received OP: 11383.09
Project: Erigon, Scaled Result: Received OP: 111.24
Project: ethers.js, Scaled Result: Received OP: 1171.71
Project: Hardhat, Scaled Result: Received OP: 602.78
Project: ETHGlobal, Scaled Result: Received OP: 0.00
Project: Vyper, Scaled Result: Received OP: 92.54
Project: DefiLlama, Scaled Result: Received OP: 392.14
Project: OpenZeppelin Contracts, Scaled Result: Received OP: 32660.92
Project: Web3js, Scaled Result: Received OP: 9418.81
Project: Dappnode, Scaled Result: Received OP: 1.16
Project: Account Abstraction - ERC-4337, Scaled Result: Received OP: 2.63
Project: 🏰 BuidlGuidl, Scaled Result: Received OP: 577.57
Project: IPFS, Scaled Result: Received OP: 16534.19
Project: beaconcha.in, Scaled Result: Received OP: 5.26
Project: Ethereum Attestation Service, Scaled Result: Received OP: 0.01
Project: L2BEAT

## Composite Calculation

In [67]:
data = pd.read_csv("RetroDistribution.csv")
data.drop(columns=['Unnamed: 0'], inplace=True)

weights = {
    'OSO: Total Stars': 0.4,
    'OSO: Total Forks': 0.3,
    'OSO: Total Contributors': 0.3
}

def normalize_data(data):
    normalized_data = {}
    for feature, values in data.items():
        if feature == "Meta: Project Name":
            continue 
        if all(isinstance(v, int) for v in values): 
            total_sum = sum(values)
            normalized_data[feature] = [v / total_sum for v in values]
        else: 
            try:
                values = [float(v) for v in values]
                max_value = max(values)
                normalized_data[feature] = [v / max_value for v in values]
            except ValueError:
                print(f"Error: Non-numeric value encountered in feature '{feature}'. Skipping normalization.")
    return normalized_data

def calculate_composite_score(data, weights):
    composite_scores = []
    for i in range(len(data)):

        composite_score = sum(data[feature][i] * weights[feature] for feature in weights)
        composite_scores.append(composite_score)
    return composite_scores

normalized_data = normalize_data(data.copy())

composite_scores = calculate_composite_score(normalized_data, weights)

for project, score in zip(data["Meta: Project Name"], composite_scores):
    print(f"Project: {project}, Composite Score: {score:.2f}")


Project: Protocol Guild, Composite Score: 1.00
Project: go-ethereum, Composite Score: 0.79
Project: Solidity, Composite Score: 0.31
Project: Erigon, Composite Score: 0.06
Project: ethers.js, Composite Score: 0.13
Project: Hardhat, Composite Score: 0.10
Project: ETHGlobal, Composite Score: 0.00
Project: Vyper, Composite Score: 0.06
Project: DefiLlama, Composite Score: 0.12
Project: OpenZeppelin Contracts, Composite Score: 0.44
Project: Web3js, Composite Score: 0.28
Project: Dappnode, Composite Score: 0.01
